# Data Loading 
This section is for loading the train and test data.

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn import tree, preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score,precision_score,recall_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline
import pandas as pd
import matplotlib.pyplot as plt # for plotting data and creating different charts.
import numpy as np # for math and arrays
import pandas as pd # data from for the data.
import seaborn as sns # for plotting.

### Load training data.
This section is for loading the training data.

In [ ]:
# Make sure to change the path to your local directory
weather_data = pd.read_csv('C:/Users/vchalla2/Downloads/ECONet/train.csv')
df = weather_data

### Load test data.
This section is for loading the test data.

In [ ]:
# Make sure to change the path to your local directory
weather_data_test = pd.read_csv('C:/Users/vchalla2/Downloads/ECONet/test.csv')
df_test = weather_data_test

# Data Preprocessing
This section is for data preprocessing.

### Preprocessing the train data.
This section is for train data preprocessing.

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
#Encoding non-traditional features
le = LabelEncoder()
scaling = MinMaxScaler()
df.Station = le.fit_transform(df.Station)
df.measure = le.fit_transform(df.measure)
df['target'] = le.fit_transform(df['target'])
#converting time series to numeric and performed encoding on station,measure 
df['Ob'] = pd.to_numeric(pd.to_datetime(df['Ob']))
df.value= scaling.fit_transform(df[['value']])

normalized = df[{'value', 'measure'}].groupby('measure').transform(lambda x: (x - x.mean()) / x.std())
df.value = normalized['value']
df

### Preprocessing the test data.
This section is for test data preprocessing.

In [ ]:
#Encoding non-traditional features
df_test.Station = le.fit_transform(df_test.Station)
df_test.measure = le.fit_transform(df_test.measure)
#converting time series to numeric and performed encoding on station,measure 
df_test['Ob'] = pd.to_numeric(pd.to_datetime(df_test['Ob']))
df_test.value = scaling.fit_transform(df_test[['value']])
normalized_test = df_test[{'value', 'measure'}].groupby('measure').transform(lambda x: (x - x.mean()) / x.std())
df_test.value = normalized_test['value']
df_test['value'].fillna(0, inplace=True)
df_test['measure'].fillna(0, inplace=True)
df_test['Ob'].fillna(0, inplace=True)
df_test

# Over Sampling Using SMOTE
This section performs oversampling using SMOTE. At the end it does the train-test split.

In [ ]:
from imblearn.over_sampling import SMOTE
X = df.drop('target',axis='columns')
y = df['target']

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)
X_train,X_test,y_train,y_test=train_test_split(X_sm,y_sm,test_size=0.3,random_state=0)

# Model Execution
This section trains the model on the train data and then performs predictions on the test data.

In [ ]:
from sklearn import tree
df_clf=tree.DecisionTreeClassifier(max_depth=10,criterion='gini')
df_clf.fit(X_train,y_train)
y_pred_test = df_clf.predict_proba(df_test)
y_pred = df_clf.predict(X_test)
y_true = y_test
# Make sure to change the directory to the local target.
pd.DataFrame(y_pred_test[:,1], columns=['target']).to_csv('C:/Users/vchalla2/Downloads/ECONet/decision_tree_smote.csv', index=False)

# Model Evaluation and Metrics.
This section evaluates the model and provides us with the metrics report.

In [ ]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from matplotlib import pyplot

confusion_matrix(y_test,y_pred)
print('Precision: %.3f' % precision_score(y_test, y_pred))
print('Recall: %.3f' % recall_score(y_test, y_pred))
print(classification_report(y_true,y_pred))
print("Accuracy: " + str( accuracy_score(y_true,y_pred)))
from sklearn.metrics import fbeta_score
print("F-2 Score: "+ str(fbeta_score(y_true,y_pred,beta=2)))
# precision-recall curve and f1
lr_precision, lr_recall, _ = precision_recall_curve(y_true,y_pred)
lr_f1, lr_auc = f1_score(y_pred, y_true), auc(lr_recall, lr_precision)
# summarize scores
print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))
# plot the precision-recall curves
no_skill = len(y_true[y_true==1]) / len(y_true)
pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
# Can input diffrent models curves just as above code
pyplot.plot(lr_recall, lr_precision, marker='.', label='LSTM')
# axis labels
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()